In [1]:
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
import time
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from collections import Counter
from IPython.display import clear_output

from scripts import BpeTokenizer, Model, Trainer, Collator, MyDataset, generate

# Загружаем данные

In [2]:
df = pd.read_csv('data/dataset.csv')
train_texts = df['text'][:-1024].tolist()
eval_texts = df['text'][-1024:].tolist()

# Инициализируем и обучаем токенизатор

In [3]:
tokenizer = BpeTokenizer()

In [4]:
tokenizer.train(train_texts[:2048], max_vocab=2048)

pair=(277, 338), freq=52: 100%|██████████| 1789/1789 [08:49<00:00,  3.38it/s]  


# Создаем датасеты и Collator

In [5]:
train_dataset = MyDataset(train_texts, tokenizer, max_length=128)
eval_dataset = MyDataset(eval_texts, tokenizer, max_length=128)
collator = Collator(tokenizer.pad_token_id)

100%|██████████| 1024/1024 [01:07<00:00, 15.21it/s]


# Создаем модель

In [6]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [8]:
model = Model(tokenizer.get_vocab_size(), emb_size=128, hidden_size=256, num_layers=2, dropout=0.1)

# Создаем Trainer и запускаем обучение

In [9]:
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    n_epochs=8,
    train_batch_size=32,
    eval_batch_size=32,
    eval_steps=64,
    collator=collator,
    lr=1e-2,
    ignore_index=tokenizer.pad_token_id
)

In [10]:
trainer.train()

epoch=0.125, loss=4.9776201248168945:   2%|▏         | 64/4096 [00:47<47:46,  1.41it/s]       

epoch=0.125, eval_loss=4.977377727627754


epoch=0.25, loss=4.256743907928467:   3%|▎         | 128/4096 [01:46<50:30,  1.31it/s]         

epoch=0.25, eval_loss=4.1699263751506805


epoch=0.375, loss=3.9351179599761963:   5%|▍         | 192/4096 [02:39<46:29,  1.40it/s]        

...

epoch=7.75, eval_loss=3.1721436232328415


epoch=7.875, loss=2.988830089569092:  98%|█████████▊| 4032/4096 [57:04<00:44,  1.44it/s]       

epoch=7.875, eval_loss=3.1678682938218117


epoch=8.0, loss=3.0986835956573486: 100%|██████████| 4096/4096 [58:11<00:00,  1.17it/s]        

epoch=8.0, eval_loss=3.162910506129265


# Оцениваем качество и проверяем жадную и случайную генерацию

In [11]:
trainer.evaluate()

3.162910506129265

In [12]:
generate(model, tokenizer, temperature=0)

'Козерогам стоит быть внимательнее к тому, что вы не будете превосходно и устали, то не сбавить свои силы и способности. В конце дня вы способны снизить свои силы и энергичные и чужие ошибки. В конце дня вы способны снизить свои силы и энергичные и чужие ошибки. В конце дня вы способны снизить свои силы и энергичные и чужие ошибки. В конце дня вы способны снизить свои силы и энергичные и чужие ошибки. В конце дня вы способны снизить свои силы и энергичные и чужие ошибки. В конце дня вы способны снизить свои силы и энергичные и чужие ошибки. В конце дня вы способны снизить свои силы и энергичные и чужие ошибки. В конце дня вы способны снизить свои силы и энергичные и чужие ошибки. В конце дня вы способны снизить свои силы и энергичные и чужие ошибки. В конце дня вы способны снизить свои силы и энергичные и чужие ошибки. В конце дня вы способны снизить свои силы и энергичные и чужие ошибки. В конце дня вы способны снизить свои силы и энергичные и чужие ошибки. В конце дня вы способны сни

In [13]:
generate(model, tokenizer, temperature=0.5, top_k=20)

'Эмоциональность, уединение, двусмысленное место работы. Во второй половине дня вероятнее утро для активного отдыха, устранения недавних событий, возможны проблемы в связи с финансовыми деловыми партнерами. Не бездельнично расслабляться в кругу близких людей или родных, друзей, родителей.'